# 4. Autres méthodes d'analyse factorielle

Dans ce notebook, nous allons utiliser d'autres méthodes d'analyse factorielle en utilisant la librairie [prince](https://github.com/MaxHalford/prince).

Cette librairie n'est probablement pas installée par défaut :

In [ ]:
!pip uninstall -y prince

In [ ]:
!pip install prince==0.7.1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import seaborn as sns
import prince

La cellule suivante est nécessaire pour assurer la compatibilité de prince 0.7.1 avec Google Colab :

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
np.float = float

## 4.1 Analyse factorielle des correspondances

Nous allons appliquer une analyse des correspondances (Correspondence Analysis) sur un jeu de données contenant un tableau de contingence du nombre de prix nobels obtenus par certains pays dans certaines catégories.

Chargez le jeu de données `nobel_data.csv` dans un dataframe en utilisant le paramètre `index_col` pour indiquer à Pandas que l'index de notre dataframe sera la colonne `Country` :

In [ ]:
df = pd.read_csv('/data/nobel_data.csv', index_col='Country')
df

Instancier la classe `CA` (les paramètres par défaut conviennent pour notre cas d'étude) et exécuter la méthode ` fit` sur la dataframe :

In [ ]:
ca = prince.CA(n_components=2)
ca.fit(df)

Visualisez les coordonnées des lignes et des colonnes avec les méthodes `row_coordinates`, et `column_coordinates` de votre objet `ca` :

In [ ]:
ca.row_coordinates(df)

In [ ]:
ca.column_coordinates(df)

Projetez ces coordonnées dans le plan obtenu avec la méthode `plot_coordinates` (vous pouvez utiliser le paramètre `figsize=(8, 8)` pour obtenir un graphique plus lisible).

Sachant que les points (aussi bien les pays que les catégories de prix Nobel) de notre tableau de contingence sont éloignés du centre de notre projection, que pouvez-vous en conclure ?

In [ ]:
ca.plot_coordinates(df, figsize=(8, 8))

L'Italie et la France semblent plus liés au prix Nobel de littérature. Le Japon et l'Allemagne à ceux de physique et de chimie. Le Royaume-Uni se rapproche du profil moyen des pays (au centre de notre projection).

Les deux cellules suivantes affichent une représentation des profils moyen des lignes et des colonnes de notre tableau de contingence. Observez-vous un lien avec le graphique précédent ?

In [ ]:
df_norm_country = df.T*100/df.T.sum()
df_norm_country['Mean profile'] = df_norm_country.mean(axis=1)
sns.heatmap(df_norm_country.T, cmap="YlGnBu", annot=True, cbar=False)

In [ ]:
df_norm_category = df*100/df.sum()
df_norm_category['Mean profile'] = df_norm_category.mean(axis=1)
sns.heatmap(df_norm_category, cmap="YlGnBu", annot=True, cbar=False)

Sur le graphique des profils moyen par pays (premier graphique), nous observons que le Royaume-Uni a un profil proche du profil moyen alors que les profils d'autres pays s'en écartent. Par exemple, la France et l'Italie ont sont surreprésentés sur les Nobels de littérature.

Nous allons maintenant appliquer cette méthode pour découvrir les correspondances pour des variables qualitatives d'un dataset contenant le résultat d'un recencement aux états-unis : [UCI Adult Dataset](https://archive.ics.uci.edu/ml/datasets/adult).

Chargez le dataset `adult-data.csv` dans une dataframe et déterminez quelles sont les variables qualitatives :

In [ ]:
df = pd.read_csv('/data/adult-data.csv')
print(df.shape)
df.head()

Créez des tableaux de contingence pour certains couples de variables qualitatives en utilisant la méthode [pandas.crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) (les deux premiers arguments de cette méthode seront des colonnes de notre dataframe, par exemple `df.education`) et réalisez une analyse factorielle des correspondances sur ces couples de variables :

In [ ]:
df_ct = pd.crosstab(df.education, df.occupation)
df_ct

In [ ]:
ca = prince.CA(n_components=2)
ca.fit(df_ct)
ca.plot_coordinates(df_ct, figsize=(12, 12))

La première composante semble liée au niveau de diplôme. Nous observons une relation entre les niveaux de diplôme et le type d'emploi occupé.

## 4.2 Analyse des correspondances multiples

Nous allons appliquer une Analyse des correspondances multiples (Multiple Correspondence Analysis) sur un jeu de données contenant des informations sur des ballons [UCI balloons dataset](https://archive.ics.uci.edu/ml/datasets/balloons).

Chargez ce dataset dans une dataframe depuis l'adresse `https://archive.ics.uci.edu/ml/machine-learning-databases/balloons/adult+stretch.data` (la méthode `read_csv` de Pandas peut charger des fichiers CSV depuis une URL) et définissez les colonnes comme étant la liste `['Color', 'Size', 'Action', 'Age', 'Inflated']` :

In [ ]:
df_balloons = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/balloons/adult+stretch.data')
df_balloons.columns = ['Color', 'Size', 'Action', 'Age', 'Inflated']
print(df_balloons.shape)
df_balloons.head()

Instancier la classe `MCA` de prince (les paramètres par défaut conviennent pour notre cas d'étude) et exécuter la méthode ` fit` sur la dataframe :

In [ ]:
mca = prince.MCA()
mca = mca.fit(df_balloons)

Projetez les individus et les variables sur les deux premières composantes principales  avec la méthode `plot_coordinates` :
* les paramètres `x_component` et `y_component` prennent des entiers en paramètres pour indiquer les composantes à utiliser
* vous pouvez utiliser le paramètre `figsize=(8, 8)` pour obtenir un graphique plus lisible
* testez les effets des paramètres booléens `show_row_points`, `show_row_labels`, `show_column_points` et `show_column_labels`
* que pouvez-vous conclure sur cette approche sur ce dataset à partir de l'inertie des composantes principales ? (affichées sur le graphique ou accessibles avec l'attribut `explained_inertia_` de votre instance de `MCA`)

In [ ]:
import matplotlib.pyplot as plt
ax = mca.plot_coordinates(
    X=df_balloons,
    figsize=(8, 8),
    show_row_points=True,
    row_points_size=10,
    show_row_labels=False,
    show_column_points=True,
    column_points_size=30,
    show_column_labels=True,
    legend_n_cols=1
)
plt.show(block=False)

Les deux premières composantes principales expliquent une bonne partie de la variabilité de notre jeu de données (40% et 12%).

Les variables affichées dans ce graphique ont été transformées en indicateurs booléens (one hot encoding), utilisez la méthode [pandas.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) pour afficher le résultat de cette transformation :

In [ ]:
pd.get_dummies(df_balloons)

Nous allons appliquer une MCA sur le dataset du recensement américain. Rechargez le si besoin, puis supprimer les colonnes numériques (`fnlwgt`, `age`, `education-num`, `hours-per-week`, `capital-gain`, `capital-loss` et `salary`) avant d'appliquer la MCA.

En fonction de l'inertie obtenue pour les premières composantes, que pouvez-vous conclure sur l'efficacité de cette approche appliquée à ce dataset ?

In [ ]:
df.head()

In [ ]:
df_cleaned_for_mca = df.drop(columns=['age', 'education-num', 'fnlwgt', 'capital-gain', 'capital-loss', 'salary', 'hours-per-week'])

In [ ]:
mca = prince.MCA(
    n_components=2,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='auto',
    random_state=42
)
mca = mca.fit(df_cleaned_for_mca)

In [ ]:
df_cleaned_for_mca.dtypes

In [ ]:
ax = mca.plot_coordinates(
    X=df_cleaned_for_mca,
    figsize=(20, 20),
    show_row_points=False,
    show_column_points=True,
    show_column_labels=True
)
plt.show(block=False)

L'inertie des deux premiers axes étant faible, cette approche sur ce jeu de données nous aide peu à comprendre nos variables.

## 4.3 Analyse factorielle des données mixtes

Nous allons appliquer une Analyse factorielle des données mixtes (Factor analysis of mixed data) sur un jeu de données d'un recensement américan.

Instancier la classe `FAMD` de prince (passer la valeur 4 pour le paramètre `n_components`, ce paramètre défini le nombre de composantes souhaitées) et exécuter la méthode ` fit` sur la dataframe :

In [ ]:
famd = prince.FAMD(n_components=70)
famd = famd.fit(df)

L'attribut `groups` de votre instane de `FAMD` indique quelles sont les attributs qualitatifs et quantitatifs de notre dataset. Vous pouvez retrouver cette information avec Pandas :
```python
num_cols = df.select_dtypes(np.number).columns.tolist()
cat_cols = list(set(df.columns) - set(num_cols))
```

In [ ]:
famd.groups

In [ ]:
num_cols = df.select_dtypes(np.number).columns.tolist()
cat_cols = list(set(df.columns) - set(num_cols))

num_cols, cat_cols

Projetez les individus sur les deux premières composantes principales  avec la méthode `plot_row_coordinates` :
* les paramètres `x_component` et `y_component` prennent des entiers en paramètres pour indiquer les composantes à utiliser
* vous pouvez utiliser le paramètre `figsize=(20, 20)` pour obtenir un graphique plus lisible
* que pouvez-vous conclure sur cette approche sur ce dataset à partir de l'inertie des composantes principales par apport à celles obtenues avec l'analyse des correspondances multiples ? (affichées sur le graphique ou accessibles avec l'attribut `explained_inertia_` de votre instance de `FAMD`)
* testez l'effet du paramètre `color_labels` en lui passant la colonne `salary` de notre dataframe d'origine. Que pouvez-vous conclure ?
* essayez d'autres couples de composantes (et éventuellement plus de composantes)

In [ ]:
ax = famd.plot_row_coordinates(
    df,
    figsize=(10, 10),
    x_component=0,
    y_component=1,
    color_labels=['Salary {}'.format(t) for t in df['salary']]
)

In [ ]:
ax = famd.plot_row_coordinates(
    df,
    figsize=(10, 10),
    x_component=0,
    y_component=1,
    color_labels=df['education']
)

In [ ]:
ax = famd.plot_row_coordinates(
    df,
    figsize=(10, 10),
    x_component=2,
    y_component=3,
    color_labels=df['education']
)

L'inertie expliquée par les deux premières composantes est nettement plus importante que pour l'ACM mais reste faible (5.8% et 4.6%). La coloration des points du jeu de données projetés fait apparaitre que la partie droite de notre projection est liée à des niveaux de salaire plus élevés.

La fonction suivante vous permet d'afficher un graphique représentant la variance expliquée par les différentes composantes principales et la variance cumulée.

Combien vous faut-il de composantes pour expliquer plus de 75% de la variance ?

In [ ]:
def display_scree_plot(famd):
    '''Display a scree plot for the pca'''

    scree = famd.explained_inertia_*100
    plt.figure(figsize=(15,8))
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("Number of principal components")
    plt.ylabel("Percentage explained variance")
    plt.title("Scree plot")
    plt.show(block=False)

In [ ]:
display_scree_plot(famd)

Il faut retenir les 62 premières composantes pour expliquer plus de 75% de la variance de notre jeu de données.

In [ ]:
famd.explained_inertia_[:62].sum()

L'attribut `V_` de votre instance de FAMD contient les vecteurs propres des composantes (i.e. : les coefficient). La fonction suivante est une adaptation à la FAMD de la fonction affichant les cercles de corrélation de l'ACP, ses paramètres sont :
* `famd` : une instance de FAMD
* `df` : la dataframe sur laquelle la FAMD a été appliquée
* `axis_ranks` : list, les indices des paires d'axes à afficher (chaque paire affichera un nouveau cercle de corrélation), exemple : [(0,1)]
* `min_variance` : seuil inférieur de variance pour les variables à conserver (au dela d'un certain nombre de variables, le graphique devient illisible)

Appliquez cette fonction sur votre FAMD avec ou sans seuil de variance et expliquez la coloration du nuage de point obtenu précédement.

In [ ]:
def display_circles(famd, df, axis_ranks, label_rotation=0, lims=None, min_variance=0.15):
    """Display correlation circles, one for each factorial plane"""

    pcs = pd.DataFrame(famd.V_, columns=famd._build_X_global(df).columns).T

    # For each factorial plane
    for d1, d2 in axis_ranks: 
        if min_variance is not None:
            fpcs = pcs[(pcs[d1] >= min_variance) | (pcs[d2] > min_variance)]
        else:
            fpcs = pcs
        labels = fpcs.index.tolist()

        # Initialise the matplotlib figure
        fig, ax = plt.subplots(figsize=(10,10))

        # Determine the limits of the chart
        xmin, xmax, ymin, ymax = -1, 1, -1, 1
#         if lims is not None :
#             xmin, xmax, ymin, ymax = lims
#         elif fpcs.shape[1] < 30 :
#             xmin, xmax, ymin, ymax = -1, 1, -1, 1
#         else :
#             xmin, xmax, ymin, ymax = min(fpcs[d1]), max(fpcs[d1]), min(fpcs[d2]), max(fpcs[d2])

        # Add arrows
        # If there are more than 30 arrows, we do not display the triangle at the end
        if fpcs.shape[0] < 30 :
            plt.quiver(np.zeros(fpcs.shape[0]), np.zeros(fpcs.shape[0]),
               fpcs[d1], fpcs[d2], 
               angles='xy', scale_units='xy', scale=1, color="grey")
            # (see the doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
        else:
            lines = [[[0,0],[x,y]] for x,y in fpcs[[d1,d2]].values]
            ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))

        # Display variable names
        if labels is not None:  
            for i,(x, y) in enumerate(fpcs[[d1,d2]].values):
                if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                    plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)

        # Display circle
        circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
        plt.gca().add_artist(circle)

        # Define the limits of the chart
        plt.xlim(xmin, xmax)
        plt.ylim(ymin, ymax)

        # Display grid lines
        plt.plot([-1, 1], [0, 0], color='grey', ls='--')
        plt.plot([0, 0], [-1, 1], color='grey', ls='--')

        # Label the axes, with the percentage of variance explained
        plt.xlabel('PC{} ({}%)'.format(d1+1, round(100*famd.explained_inertia_[d1],1)))
        plt.ylabel('PC{} ({}%)'.format(d2+1, round(100*famd.explained_inertia_[d2],1)))

        plt.title("Correlation Circle (PC{} and PC{})".format(d1+1, d2+1))
        plt.show(block=False)

In [ ]:
display_circles(famd, df, [(0,1)], min_variance=0)

In [ ]:
display_circles(famd, df, [(0,1)], min_variance=0.20)

In [ ]:
display_circles(famd, df, [(2,3)], min_variance=0.20)

En choisissant la valeur 0.2 pour l'attribut `min_variance`, nous observons plus clairement que les attributs `hours-per-week` et `education-num` ont un impact positif sur la première composante. Le niveau de salaire est donc corrélé aux nombres d'heures travaillées et au niveau d'éducation.

La cellule suivante permet de visualiser les coefficients des vecteurs propres de nos composantes :

In [ ]:
print(famd.V_.shape)
famd._build_X_global(df).columns
df_comp = pd.DataFrame(famd.V_, columns=famd._build_X_global(df).columns).T
df_comp

## 4.4 Application au dataset Titanic

Le fichier `titanic.csv` contient des données qualitatives et quantitatives sur certains passagers du titanic ainsi qu'une classe booléenne indiquant s'ils ont survécu ou non (`Survived`). La colonne d'index est `PassengerId`.

Utilisez les méthodes précédentes pour analyser ce dataset.

Attention : la variable `Name` n'est probablement pas utile. Certaines variables contiennent des valeurs manquantes, vous pouvez les laissez tels quels ou affecter la moyenne ou le mode ([pandas.DataFrame.mode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html)) des colonnes pour ces individus.


Voici une définition des variables :
* Survived: Survival (0 = No, 1 = Yes)
* Pclass: Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
* Name: name
* Sex: Sex	
* Age: Age in years	
* Sibsp: # of siblings / spouses aboard the Titanic	
* Parch: # of parents / children aboard the Titanic	
* Ticket: Ticket number	
* Fare: Passenger fare	
* Cabin: Cabin number	
* Embarked: Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [ ]:
df = pd.read_csv('/data/titanic.csv', index_col='PassengerId')
print(df.shape)
df.head()

Nous nous intéressons aux valeurs nulles de notre dataset :

In [ ]:
df.isna().any()

La variable `Age` contient 177 valeurs nulles, nous pouvons les remplacer par l'âge moyen :

In [ ]:
df.Age.hist()

In [ ]:
df.Age.value_counts(dropna=False)

In [ ]:
df['Age'] = df.Age.fillna(value=df.Age.mean())

La variable `Cabin` contient 687 valeurs nulles (sur 891 individus), de plus elle possèdent un trop grand nombre de modalités, il est préférable de ne pas en tenir compte :

In [ ]:
df.Cabin.value_counts(dropna=False)

In [ ]:
df.drop(columns='Cabin', inplace=True)

La variable `Embarked` ne contient que deux valeurs nulles :

In [ ]:
df.Embarked.value_counts(dropna=False)

Cette variable semble corrélée à la variable `Pclass`, nous pouvons donc imputer la valeur `C` pour la variable `Embarked` de ces deux individus :

In [ ]:
display('Embarked == S')
display(df[df.Embarked == 'S'].describe())
display('Embarked == C')
display(df[df.Embarked == 'C'].describe())
display('Embarked == Q')
display(df[df.Embarked == 'Q'].describe())

In [ ]:
df[df.Embarked.isna()]

In [ ]:
df['Embarked'] = df.Embarked.fillna('C')

In [ ]:
df[df.Embarked.isna()].describe()

Finalement, la variable `Name` ne serait pas utile à moins d'appliquer un pré-traitement (tel que retrouver des liens de parentés en se fiant au nom de famille) :

In [ ]:
df_cleaned = df.drop(columns=['Name', 'Ticket', 'Survived'])

Nous pouvons ensuite appliquer une AFDM et visualiser nos individus et nos variables sur le premier plan factoriel :

In [ ]:
famd = prince.FAMD(n_components=70)
famd = famd.fit(df_cleaned)

In [ ]:
ax = famd.plot_row_coordinates(
    df_cleaned,
    figsize=(10, 10),
    x_component=0,
    y_component=1,
    color_labels=['Survived {}'.format(t) for t in df['Survived']]
)

In [ ]:
display_circles(famd, df_cleaned, [(0,1)], min_variance=0.2)

La première composante semble liée à la survie des passagers. Cette composante est positivement liée au prix du billet et aux femmes et négativement lié à la classe de la cabine. Cette analyse confirme ce que connaissons des survivants du Titanic : des femmes issues de milieux favorisés.